## Build model

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [2]:
tokenizer = AutoTokenizer.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

In [4]:
def predict_proba(model, tokenizer, text):
    
    encoding = tokenizer.encode_plus(text, return_tensors="pt") 
    
    result = model(**encoding)

    list_results = softmax(result[0].detach().numpy()[0]).tolist()
  
    return list_results

In [5]:
def predict_proba_batch(model, tokenizer, texts):
    
    encodings = [
        tokenizer.encode_plus(
            text, return_tensors="pt"
        ) for text in texts]
    
    results = [
        model(**encoding) for encoding in encodings
    ]
    
    list_results = [
        softmax(result[0].detach().numpy()[0]).tolist()
        for result in results
    ]
        
    return list_results, ['negative', 'positive'], [0, 1]

In [6]:
predict_proba(model, tokenizer, "hi")

[0.0016732359072193503, 0.9983267784118652]

In [7]:
predict_proba_batch(model, tokenizer, ["hi", "bye"])

([[0.0016732359072193503, 0.9983267784118652],
  [0.9684281349182129, 0.031571850180625916]],
 ['negative', 'positive'],
 [0, 1])

# Unbox

### Pack model

In [8]:
import unboxapi
client = unboxapi.UnboxClient()

User is not logged in.


In [9]:
saved_path = client.pack_model(
    function=predict_proba_batch, 
    model=model,
    tokenizer=tokenizer,
    model_name="TransformersModel",
    model_type="transformers"
)

[2021-05-12 14:46:04,423] INFO - BentoService bundle 'TransformersModel:20210512144603_12CC1E' saved to: /Users/gbayomi/bentoml/repository/TransformersModel/20210512144603_12CC1E


### Load

In [10]:
import bentoml
from pandas import DataFrame

In [11]:
bento_model = bentoml.load(saved_path)

[2021-05-12 14:46:05,643] WARNING - Module `template_model` already loaded, using existing imported module.


### Test locally

In [12]:
bento_model.predict([{"text": "great"}])

[{'negative': 0.0001369865203741938, 'positive': 0.9998631477355957}]

In [13]:
bento_model.predictbatch([{"batch": ["great", "terrible"]}])

[[{'negative': 0.0001369865203741938, 'positive': 0.9998631477355957},
  {'negative': 0.9997530579566956, 'positive': 0.0002470429753884673}]]

In [14]:
bento_model.predictactive([{"batch": ["great", "terrible"], "n_instances": 2}])

[([1, 0], ['terrible', 'great'])]

### Test as an endpoint

In [15]:
!bentoml run TransformersModel:latest predict --input '{"text": "Which baking dish is best to bake a banana bread ?"}'

[2021-05-12 14:46:13,318] INFO - Getting latest version TransformersModel:20210512144603_12CC1E
[2021-05-12 14:46:15,977] INFO - {'service_name': 'TransformersModel', 'service_version': '20210512144603_12CC1E', 'api': 'predict', 'task': {'data': '{"text": "Which baking dish is best to bake a banana bread ?"}', 'task_id': '8a766a46-f614-43be-9995-8cd17f14899f', 'cli_args': ('--input', '{"text": "Which baking dish is best to bake a banana bread ?"}'), 'inference_job_args': {}}, 'result': {'data': '{"negative": 0.9985238313674927, "positive": 0.0014760877238586545}', 'http_status': 200, 'http_headers': (('Content-Type', 'application/json'),)}, 'request_id': '8a766a46-f614-43be-9995-8cd17f14899f'}
{"negative": 0.9985238313674927, "positive": 0.0014760877238586545}


In [16]:
!bentoml run TransformersModel:latest predictbatch --input '{"batch": ["great", "terrible"]}'

[2021-05-12 14:46:18,030] INFO - Getting latest version TransformersModel:20210512144603_12CC1E
[2021-05-12 14:46:20,402] INFO - {'service_name': 'TransformersModel', 'service_version': '20210512144603_12CC1E', 'api': 'predictbatch', 'task': {'data': '{"batch": ["great", "terrible"]}', 'task_id': '5256ca7f-bf98-4526-bcec-a55cfeb33341', 'cli_args': ('--input', '{"batch": ["great", "terrible"]}'), 'inference_job_args': {}}, 'result': {'data': '[{"negative": 0.0001369865203741938, "positive": 0.9998631477355957}, {"negative": 0.9997530579566956, "positive": 0.0002470429753884673}]', 'http_status': 200, 'http_headers': (('Content-Type', 'application/json'),)}, 'request_id': '5256ca7f-bf98-4526-bcec-a55cfeb33341'}
[{"negative": 0.0001369865203741938, "positive": 0.9998631477355957}, {"negative": 0.9997530579566956, "positive": 0.0002470429753884673}]


In [17]:
!bentoml run TransformersModel:latest predictactive --input '{"batch": ["great", "terrible"], "n_instances": 2}'

[2021-05-12 14:46:22,450] INFO - Getting latest version TransformersModel:20210512144603_12CC1E
[2021-05-12 14:46:24,895] INFO - {'service_name': 'TransformersModel', 'service_version': '20210512144603_12CC1E', 'api': 'predictactive', 'task': {'data': '{"batch": ["great", "terrible"], "n_instances": 2}', 'task_id': 'b938f3c8-f6df-48e7-a13b-5e3ca05f7028', 'cli_args': ('--input', '{"batch": ["great", "terrible"], "n_instances": 2}'), 'inference_job_args': {}}, 'result': {'data': '[[1, 0], ["terrible", "great"]]', 'http_status': 200, 'http_headers': (('Content-Type', 'application/json'),)}, 'request_id': 'b938f3c8-f6df-48e7-a13b-5e3ca05f7028'}
[[1, 0], ["terrible", "great"]]
